In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import sys
from tqdm import tqdm
sys.path.insert(0, os.getcwd() + '/code')
from viz import *
from helper_functions import * 
from multiprocessing import Pool
from scipy.signal import argrelextrema

In [2]:
df_game = pd.read_csv("data/games.csv")
df_tracking = pd.read_csv("data/week1.csv")
df_play = pd.read_csv("data/plays.csv")
df_players = pd.read_csv("data/players.csv")
df_pffScoutingData = pd.read_csv("data/pffScoutingData.csv")

df_tracking = pd.read_csv(f"data/week{1}.csv")
df_area_t = pd.read_csv(f"data/area/area_t.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1'])
df_area_features = pd.read_csv(f"data/area_features/Area_features.csv").drop(columns=['Unnamed: 0'])
#playIds = df_area.playId.unique()

In [3]:
print(df_area_t)

        frameId       Area  playId      gameId
0             1  22.319900      97  2021090900
1             2  22.162750      97  2021090900
2             3  22.160150      97  2021090900
3             4  22.453350      97  2021090900
4             5  22.727850      97  2021090900
...         ...        ...     ...         ...
361365       54   9.889471    4433  2021110100
361366       55  11.820654    4433  2021110100
361367       56  12.093260    4433  2021110100
361368       57  12.366510    4433  2021110100
361369       58   7.808204    4433  2021110100

[361370 rows x 4 columns]


In [4]:
print(df_area_features)

      playId      gameId     event  te         Ae  tc         Ac
0         97  2021090900      pass  40  12.237518  37  15.013121
1        137  2021090900      pass  32  16.305513  22  26.760950
2        187  2021090900      pass  28  18.312900  20  25.935250
3        282  2021090900      pass  38  29.901092  35  36.001815
4        349  2021090900      pass  33  14.784734  19  24.298700
...      ...         ...       ...  ..        ...  ..        ...
7569    4310  2021110100  scramble  40  27.696650  22  33.668850
7570    4363  2021110100  scramble  41  12.143444  36  17.151700
7571    4392  2021110100      sack  48  13.393264  44  15.240555
7572    4411  2021110100      pass  33  12.437731  19  25.642100
7573    4433  2021110100  scramble  53   9.259213  39   8.102456

[7574 rows x 7 columns]


In [5]:
print(f'Total n° of plays : {len(df_play)}')
scramble_mask = "(event == 'scramble' | event == 'sack')"
scramble_plays = df_area_features.query(scramble_mask)
scramble_plays = pd.merge(scramble_plays,df_play,how="left",on = ["playId","gameId"])
print(f'n° of scramble or sack plays : {len(scramble_plays)}')
scramble_plays_wo_PA = scramble_plays[scramble_plays.pff_playAction == 0]
print(f'n° of scramble or sack plays w/o play action : {len(scramble_plays_wo_PA)}')
print(scramble_plays_wo_PA)


Total n° of plays : 8557
n° of scramble or sack plays : 1815
n° of scramble or sack plays w/o play action : 1139
      playId      gameId     event  te         Ae  tc         Ac  \
2        947  2021090900  scramble  37   6.471160  18  23.764250   
6       2032  2021090900  scramble  38  13.483074  33  17.202300   
7       2279  2021090900  scramble  35   8.196493  17  21.519200   
8       2298  2021090900  scramble  41   4.706347  23  27.397200   
9       2330  2021090900  scramble  42  23.306400  22  24.664650   
...      ...         ...       ...  ..        ...  ..        ...   
1810    4049  2021110100  scramble  31  39.334050  25  45.328300   
1811    4310  2021110100  scramble  40  27.696650  22  33.668850   
1812    4363  2021110100  scramble  41  12.143444  36  17.151700   
1813    4392  2021110100      sack  48  13.393264  44  15.240555   
1814    4433  2021110100  scramble  53   9.259213  39   8.102456   

                                        playDescription  quarter  down

In [6]:
scramble_plays["Acrit"] = np.nan
for scramble_id in tqdm(scramble_plays.index):

    scramble_play = scramble_plays.loc[scramble_id].playId
    scramble_game = scramble_plays.loc[scramble_id].gameId

    selected_area_features_df = df_area_features[(df_area_features .playId==scramble_play)&(df_area_features.gameId==scramble_game)].copy()
    selected_area_t_df = df_area_t[(df_area_t.playId==scramble_play)&(df_area_t.gameId==scramble_game)].copy()

    Acrit = np.min(selected_area_t_df[selected_area_t_df.frameId <= int(selected_area_features_df.te)].Area)
    scramble_plays.loc[scramble_id,"Acrit"] = Acrit
print(scramble_plays)
print(scramble_plays.Acrit.mean())
print(scramble_plays.Acrit.std())


100%|██████████| 1815/1815 [00:07<00:00, 242.91it/s]

      playId      gameId     event  te         Ae  tc         Ac  \
0        735  2021090900  scramble  40  49.791600   6  35.893200   
1        764  2021090900  scramble  28  24.175115  25  26.277012   
2        947  2021090900  scramble  37   6.471160  18  23.764250   
3       1312  2021090900  scramble  24  28.861884  24  28.861884   
4       1425  2021090900  scramble  31  32.751879  25  42.508076   
...      ...         ...       ...  ..        ...  ..        ...   
1810    4049  2021110100  scramble  31  39.334050  25  45.328300   
1811    4310  2021110100  scramble  40  27.696650  22  33.668850   
1812    4363  2021110100  scramble  41  12.143444  36  17.151700   
1813    4392  2021110100      sack  48  13.393264  44  15.240555   
1814    4433  2021110100  scramble  53   9.259213  39   8.102456   

                                        playDescription  quarter  down  ...  \
0     (2:45) (Shotgun) D.Prescott pass short right t...        1     2  ...   
1     (2:22) (No Huddle) 

In [7]:
scramble_plays_wo_PA["Acrit"] = np.nan
print(len(scramble_plays_wo_PA))
for scramble_id in tqdm(scramble_plays_wo_PA.index):

    scramble_play = scramble_plays_wo_PA.loc[scramble_id].playId
    scramble_game = scramble_plays_wo_PA.loc[scramble_id].gameId

    selected_area_features_df = df_area_features[(df_area_features .playId==scramble_play)&(df_area_features.gameId==scramble_game)].copy()
    selected_area_t_df = df_area_t[(df_area_t.playId==scramble_play)&(df_area_t.gameId==scramble_game)].copy()

    Acrit = np.min(selected_area_t_df[selected_area_t_df.frameId <= int(selected_area_features_df.te)].Area)
    scramble_plays_wo_PA.loc[scramble_id,"Acrit"] = Acrit
print(scramble_plays_wo_PA)
print(scramble_plays_wo_PA.Acrit.mean())
print(scramble_plays_wo_PA.Acrit.std())

/tmp/ipykernel_139257/2300602930.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1139


100%|██████████| 1139/1139 [00:05<00:00, 222.85it/s]

      playId      gameId     event  te         Ae  tc         Ac  \
2        947  2021090900  scramble  37   6.471160  18  23.764250   
6       2032  2021090900  scramble  38  13.483074  33  17.202300   
7       2279  2021090900  scramble  35   8.196493  17  21.519200   
8       2298  2021090900  scramble  41   4.706347  23  27.397200   
9       2330  2021090900  scramble  42  23.306400  22  24.664650   
...      ...         ...       ...  ..        ...  ..        ...   
1810    4049  2021110100  scramble  31  39.334050  25  45.328300   
1811    4310  2021110100  scramble  40  27.696650  22  33.668850   
1812    4363  2021110100  scramble  41  12.143444  36  17.151700   
1813    4392  2021110100      sack  48  13.393264  44  15.240555   
1814    4433  2021110100  scramble  53   9.259213  39   8.102456   

                                        playDescription  quarter  down  ...  \
2     (:05) (Shotgun) T.Brady pass incomplete short ...        1     3  ...   
6     (2:45) (No Huddle, 

In [10]:
print(f'Total n° of plays : {len(df_play)}')
pass_mask = "event == 'pass'"
pass_plays = df_area_features.query(pass_mask)
pass_plays = pd.merge(pass_plays,df_play,how="left",on = ["playId","gameId"])
print(f'n° of pass plays : {len(pass_plays)}')
pass_plays_wo_PA = pass_plays[pass_plays.pff_playAction == 0]
print(f'n° of scramble or sack plays w/o play action : {len(pass_plays_wo_PA)}')
print(pass_plays_wo_PA)

Total n° of plays : 8557
n° of pass plays : 5759
n° of scramble or sack plays w/o play action : 4633
      playId      gameId event  te         Ae  tc         Ac  \
0         97  2021090900  pass  40  12.237518  37  15.013121   
1        137  2021090900  pass  32  16.305513  22  26.760950   
2        187  2021090900  pass  28  18.312900  20  25.935250   
4        349  2021090900  pass  33  14.784734  19  24.298700   
5        410  2021090900  pass  22  15.032565  16  21.957300   
...      ...         ...   ...  ..        ...  ..        ...   
5752    3392  2021110100  pass  34  11.988250  20  32.804600   
5754    3572  2021110100  pass  39   6.898700  20  34.447850   
5756    4071  2021110100  pass  38  15.314200  24  33.071150   
5757    4113  2021110100  pass  33  19.062700  21  29.345050   
5758    4411  2021110100  pass  33  12.437731  19  25.642100   

                                        playDescription  quarter  down  ...  \
0     (13:33) (Shotgun) T.Brady pass incomplete dee

In [11]:
print(pass_plays.Ae.mean())
print(pass_plays.Ae.std())

print(pass_plays_wo_PA.Ae.mean())
print(pass_plays_wo_PA.Ae.std())

19.73143116204909
11.913234431826725
17.173106886110627
7.778071511161969
